In [1]:
import pandas as pd
import xlrd
import tushare as ts
import numpy as np
import os
import talib

In [41]:
token = "7845848405417df2eec8b602de4f1c07766c33fe2c0f4a41d912f4a5"
ts.set_token(token)
pro = ts.pro_api()

In [42]:
# 计算DB1指数
def calculateDB1(HIGH, LOW, CLOSE, M1, M2, smaArray=[]):
    MIN_21 = []
    MAX_21 = []
    for i in range(0, 21):
        MIN_21.append(LOW[:i + 1].min())
        MAX_21.append(HIGH[:i + 1].max())
    for i in range(21, len(LOW)):
        MIN_21.append(LOW[i - 20:i + 1].min())
        MAX_21.append(HIGH[i - 20:i + 1].max())
    MIN_21 = np.array(MIN_21)
    MAX_21 = np.array(MAX_21)
    # DB1赋值: (收盘价 - 21日内最低价的最低值) / (21日内最高价的最高值-21日内最低价的最低值) * 100
    smaArray = (CLOSE - MIN_21) / (MAX_21 - MIN_21) * 100
    return MIN_21, MAX_21, smaArray

In [43]:
# 计算DB2指数
def calculateDB2(DB1, M1, M2, smaArray=[]):
    length = len(DB1)
    if not smaArray:
        # 首日EMA記為當日收盤價
        smaArray.append(DB1[0])
        for i in range(1, length):
            ema = (M2 * DB1[i] + (M1 - M2) * smaArray[-1]) / M1
            smaArray.append(ema)
    return smaArray

In [51]:
# ========== 创建汇总表模板 ==========
def buildTableHead(start_time2):
    # ========== 1 日前复权数据 DB2 ==========
    demo1 = ts.pro_bar('600000.SH', adj='qfq', start_date=start_time2)
    demo = demo1.sort_values('trade_date', ascending=True)  # sort
    demo.index = demo1.index
    
    demoT = demo['trade_date'].T  # get column names
    demoT = demoT[-5:]  # slice
    
    dateName = demoT.tolist()  # df -> list
    dateName = ['DB2_day_' + i for i in dateName]  # add index label
    
    # ========== 2 周前复权数据 DB2 ==========
    demo_week1 = ts.pro_bar('600000.SH', freq='W', adj='qfq', start_date=start_time2)
    demo_week = demo_week1.sort_values('trade_date', ascending=True)
    demo_week.index = demo_week1.index
    
    demo_weekT = demo_week['trade_date'].T
    demo_weekT = demo_weekT[-5:]
    weekName = demo_weekT.tolist()
    weekName = ['DB2_week_' + i for i in weekName]
    
    # ========== 3 MACD_hist ==========
    macdName = demoT.tolist()  # df -> list
    macdName = ['MACD_' + i for i in macdName]  # add index label
    
    # ========== 4 RSI ==========
    RSIName = demoT.tolist()  # df -> list
    RSIName = ['RSI_' + i for i in RSIName]  # add index label
    
    # ========== 5 CCI ==========
    CCIName = demoT.tolist()  # df -> list
    CCIName = ['CCI_' + i for i in CCIName]  # add index label
    
    # ========== 6 KDJ ==========
    KDJName = demoT.tolist()  # df -> list
    KDJName = ['KDJ_' + i for i in KDJName]  # add index label
    
    # ========== 7 企业基本数据 ==========
    empl = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
    emplName = empl.columns.tolist()
    
    # ========== 8.汇总表头名称 ==========
    colsName = dateName + weekName + macdName + RSIName + CCIName + KDJName + emplName
    conclude = pd.DataFrame(columns=colsName)
    
    return conclude

In [52]:
# ========== Initialize ==========
# 输入文件路径
path = "沪深A股.xls"

# 输入要读取的工作表 / sheet 数，默认读取第一张
# sheets = [5]
sheets = [0, 1, 2, 3, 4]

# 输入储存文件路径
savePath = "./data"

# 设置计算 ADX、PDI 和 MDI 的周期，默认为14
period = 14

# 设置计算 SMA 的周期 N
N = 13
N2 = 8
weight = 1

# 设置起始时间，两个要保持一致
start_time = '2019-8-05'
start_time2 = '20190805'

In [53]:
conclude = buildTableHead(start_time2)

In [54]:
conclude

,DB2_day_20210825,DB2_day_20210826,DB2_day_20210827,DB2_day_20210830,DB2_day_20210831,DB2_week_20210730,DB2_week_20210806,DB2_week_20210813,DB2_week_20210820,DB2_week_20210827,...,KDJ_20210826,KDJ_20210827,KDJ_20210830,KDJ_20210831,ts_code,symbol,name,area,industry,list_date
